In [195]:
import numpy as np
import time
import os

In [196]:
current_path = os.getcwd()
filename = os.path.join(current_path, "matrix_100x100.csv")
matrix1 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix1),np.array_equal(matrix1, matrix1.T))

filename = os.path.join(current_path, "matrix_200x200.csv")
matrix2 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix2),np.array_equal(matrix2, matrix2.T))

filename = os.path.join(current_path, "matrix_400x400.csv")
matrix4 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix4),np.array_equal(matrix4, matrix4.T))

filename = os.path.join(current_path, "matrix_800x800.csv")
matrix8 = np.genfromtxt(filename, delimiter=',',).astype(np.float32)
print(np.shape(matrix8),np.array_equal(matrix8, matrix8.T))

(100, 100) True
(200, 200) True
(400, 400) True
(800, 800) True


In [197]:
EPSILON = 1e-8  # Using exponentiation
MAX_ITERATIONS = 1000

In [198]:
def norm(v):
    return np.sqrt(np.dot(v,v))

def normalize(v):
    return v/norm(v)

In [199]:
def qr_householder(A):
    m, n = A.shape
    R = A.astype(float)  # Avoid unnecessary copy, ensure mutability
    Q = np.eye(m)

    for i in range(n - 1):
        # Compute the Householder vector
        u = R[i:, i].copy()  # Work with a slice; make a copy to modify
        alpha = norm(u)
        u[0] -= alpha  # Create the Householder vector in place
        norm_u = norm(u)

        if norm_u != 0:  # Avoid division by zero
            u /= norm_u

            # Apply the reflection to R in place
            R[i:, i:] -= 2 * np.outer(u, u @ R[i:, i:])
            
            # Apply the reflection to Q in place
            Q[:, i:] -= 2 * np.outer(Q[:, i:] @ u, u)
    return Q, R


In [260]:
def sum_lower_triangle(R, epsilon):
    rows, cols = R.shape
    sum = 0
    for i in range(1, rows):  # Start from row 1 (skip diagonal)
        for j in range(i):  # Only check lower triangle
            if (R[i, j] > epsilon):
                return epsilon + 100000000
    return sum

In [254]:
def qr_algorithm_with_shifts(A, refeig=-1, sindex=-1, max_iterations=1000, epsilon = 1e-8):
    # Compute the initial QR decomposition of A
    
    Ai = A
    Q_total = np.eye(A.shape[0])
    
    
    for i in range(max_iterations):
        shift = Ai[refeig, sindex]
        B = Ai.copy()
        
        for j in range(B.shape[0]): # Subtract the shift from the diagonal
            B[j,j] -= shift
        
        Q, R = qr_householder(B)
 
        Ai = R @ Q
        for j in range(Ai.shape[0]): # Add the shift back to the diagonal
            Ai[j,j] += shift

        # Compute the total Q matrix (Q1 * Q2 * ... * Qi)
        Q_total =  Q_total @ Q

        convergence = sum_lower_triangle(Ai, epsilon)
        print(f"Iteration: {i} The dconvergence rate is: {convergence}")
        
        if (convergence < epsilon):
            print(f"The matrix R is converged: \n{R}")
            break
        
    #Return the diagnol of the matrix as eigen values and the total Q matrix as eigen vectors
    return Ai.diagonal(), Q_total 



In [202]:
def qr_algorithm_recursive(A, max_iterations=1000, epsilon = 1e-8, iterations = 0):
    # Compute the initial QR decomposition of A
    if (iterations >= max_iterations):
        throw("Max iterations reached")
    
    Q, R = np.linalg.qr(A)
    Aj = A # Save the previous iteration matrix
    Ai = R @ Q
    diff = norm(Aj.diagonal() - Ai.diagonal())
    print(f"Iteration: {iterations} The diff is: {diff}")
    
    if (is_converged(Ai, epsilon) and diff < epsilon):
        return Ai.diagonal(), Ai
    else:
        diag, Qt = qr_algorithm_recursive(Ai, max_iterations, epsilon, iterations+1)
    return diag, Qt @ Q

In [203]:
# def is_converged(A, epsilon):
#     """Check if the lower triangular part of A is close to zero."""
#     return np.all(np.abs(np.tril(A, k=-1)) < epsilon)

# def qr_algorithm_recursive(A, max_iterations=1000, epsilon=1e-8, iterations=0, Q_accum=None):
#     if iterations >= max_iterations:
#         raise Exception("Max iterations reached")

#     Q, R = np.linalg.qr(A)  # Compute QR decomposition

#     if Q_accum is None:
#         Q_accum = Q  # Initialize Q accumulation
#     else:
#         Q_accum = Q_accum @ Q  # Accumulate Q for eigenvectors

#     Ai = R @ Q  # Compute next matrix
#     diff = norm(A.diagonal() - Ai.diagonal())

#     #print(f"Iteration {iterations}: diff = {diff}")

#     if is_converged(Ai, epsilon) and diff < epsilon:
#         return Ai.diagonal(), Q_accum  # Eigenvalues and accumulated eigenvectors

#     return qr_algorithm_recursive(Ai, max_iterations, epsilon, iterations+1, Q_accum)

In [258]:
matrix = matrix8

In [259]:
# Measure time for qr_shifts
start_time = time.time()
eigs_qr, eigv_qr = qr_algorithm_with_shifts(matrix, max_iterations=MAX_ITERATIONS, epsilon = EPSILON)
qr_time = time.time() - start_time
print(f"\nExecution time for qr_shifts: {qr_time:.6f} seconds")

# Measure time for NumPy's eig function
start_time = time.time()
eigs_np, eigv_np = np.linalg.eig(matrix)
np_time = time.time() - start_time
print(f"Execution time for np.linalg.eig: {np_time:.6f} seconds")

Iteration: 0 The dconvergence rate is: 47493.031653686354
Iteration: 1 The dconvergence rate is: 40049.472992470735
Iteration: 2 The dconvergence rate is: 35892.13171099673
Iteration: 3 The dconvergence rate is: 32826.90342868937
Iteration: 4 The dconvergence rate is: 30386.968320916072
Iteration: 5 The dconvergence rate is: 28395.383680247778
Iteration: 6 The dconvergence rate is: 26728.56087963264
Iteration: 7 The dconvergence rate is: 25304.349884641226
Iteration: 8 The dconvergence rate is: 24068.415548844914
Iteration: 9 The dconvergence rate is: 22986.43327655184
Iteration: 10 The dconvergence rate is: 22027.238662784344
Iteration: 11 The dconvergence rate is: 21169.519891370517
Iteration: 12 The dconvergence rate is: 20399.612819803388
Iteration: 13 The dconvergence rate is: 19703.398082685548
Iteration: 14 The dconvergence rate is: 19072.695299463616
Iteration: 15 The dconvergence rate is: 18489.490287051718
Iteration: 16 The dconvergence rate is: 17946.768973124043
Iteration: 

KeyboardInterrupt: 

In [256]:
print(f"\nEigenvalues from qr_shifts:\n{eigs_qr}")
print(f"Eigenvalues from np.linalg.eig:\n{eigs_np}")
print(f"\nEigenvectors from qr_shifts:\n{eigv_qr}")
print(f"Eigenvectors from np.linalg.eig:\n{eigv_np}")

print(f"\nEigenvectors difference: {np.linalg.norm(eigv_qr) - np.linalg.norm(eigv_np)}")



Eigenvalues from qr_shifts:
[-0.16424794  4.39138238  2.77286556]
Eigenvalues from np.linalg.eig:
[-0.16424794  2.77286556  4.39138238]

Eigenvectors from qr_shifts:
[[ 0.2260912  -0.48280128 -0.84604119]
 [ 0.6611152  -0.56181831  0.49727948]
 [-0.7154086  -0.6717612   0.19216509]]
Eigenvectors from np.linalg.eig:
[[ 0.2260912  -0.84604119 -0.48280128]
 [ 0.6611152   0.49727948 -0.56181831]
 [-0.7154086   0.19216509 -0.6717612 ]]

Eigenvectors difference: 4.440892098500626e-16


In [257]:
es = np.array(eigs_qr)
ev = np.array(eigv_qr)
print(matrix @ ev - ev * es)
print()
print(matrix @ eigv_np - eigv_np * eigs_np)

[[ 3.75914742e-09 -1.76402226e-09 -2.08322248e-12]
 [ 4.37438517e-09 -5.14537479e-09 -2.42272868e-12]
 [ 5.23041051e-09  5.57108004e-09 -2.89723801e-12]]

[[ 8.95117314e-16 -4.44089210e-16 -4.44089210e-16]
 [ 0.00000000e+00  2.22044605e-16 -8.88178420e-16]
 [ 7.91033905e-16 -1.11022302e-16  0.00000000e+00]]
